In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("transfusion_10%.csv")

In [3]:
df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,NaN,98.0,1.0
1,0.0,13.0,3250.0,28.0,1.0
2,1.0,16.0,4000.0,NaN,1.0
3,2.0,20.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,23.0,2.0,500.0,NaN,0.0
744,NaN,2.0,NaN,52.0,0.0
745,NaN,NaN,NaN,62.0,0.0
746,39.0,1.0,250.0,NaN,0.0


In [4]:
df.dtypes

Recency (months)                              float64
Frequency (times)                             float64
Monetary (c.c. blood)                         float64
Time (months)                                 float64
whether he/she donated blood in March 2007    float64
dtype: object

In [5]:
df.isnull().sum()

Recency (months)                              107
Frequency (times)                              78
Monetary (c.c. blood)                          94
Time (months)                                  95
whether he/she donated blood in March 2007      0
dtype: int64

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 3740
nilai hilang : 374
persentase : 10.0


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Recency (months)                              14.304813
Frequency (times)                             10.427807
Monetary (c.c. blood)                         12.566845
Time (months)                                 12.700535
whether he/she donated blood in March 2007     0.000000
dtype: float64

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,7)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
        
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [11]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
0,2.0,50.0,0.0,98.0
1,0.0,13.0,3250.0,28.0
2,1.0,16.0,4000.0,0.0
3,2.0,20.0,5000.0,45.0
4,1.0,24.0,6000.0,77.0
...,...,...,...,...
743,23.0,2.0,500.0,0.0
744,0.0,2.0,0.0,52.0
745,0.0,0.0,0.0,62.0
746,39.0,1.0,250.0,0.0


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 379


In [13]:
euc_dist = {}

In [14]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,7)

Kolom : 1
[2.0, 4.0, 1.0, 2.0, 14.0, 21.0, 12.0]
mean :  8.0
standar deviasi :  7.151423434733616
fuzzy :  [(2.0, 0.7033372200674907), (4.0, 0.8552098836329063), (1.0, 0.6194021072596377), (2.0, 0.7033372200674907), (14.0, 0.7033372200674907), (21.0, 0.1916540555539238), (12.0, 0.8552098836329063)]
fuzzy :  None
pembilang :  30.987565373233373
penyebut :  4.631487590281846
rata-rata tertimbang :  6.690629040711225
 
[4.0, 0.0, 4.0, 9.0, 11.0, 9.0, 11.0]
mean :  6.857142857142857
standar deviasi :  3.9070840946220984
fuzzy :  [(4.0, 0.7654036355922553), (0.0, 0.2143924066826014), (4.0, 0.7654036355922553), (9.0, 0.8603762587038636), (11.0, 0.5700065154228655), (9.0, 0.8603762587038636), (11.0, 0.5700065154228655)]
fuzzy :  None
pembilang :  34.15014508071063
penyebut :  4.605965226120571
rata-rata tertimbang :  7.4143297667650865
 
[11.0, 1.0, 5.0, 6.0, 11.0, 2.0, 3.0]
mean :  5.571428571428571
standar deviasi :  3.7742413753208304
fuzzy :  [(11.0, 0.35548340739998485), (1.0, 0.48025008

[0.0, 2.0, 2.0, 2.0, 3.0, 4.0, 4.0]
mean :  2.4285714285714284
standar deviasi :  1.2936264483053452
fuzzy :  [(0.0, 0.17169898715226825), (2.0, 0.9466059661735403), (2.0, 0.9466059661735403), (2.0, 0.9466059661735403), (3.0, 0.9070562074394016), (4.0, 0.4781989693223469), (4.0, 0.4781989693223469)]
fuzzy :  None
pembilang :  12.226396173938221
penyebut :  4.874971031756984
rata-rata tertimbang :  2.507993605355172
 
[4.0, 4.0, 11.0, 11.0, 13.0, 2.0, 0.0]
mean :  6.428571428571429
standar deviasi :  4.746642207381757
fuzzy :  [(4.0, 0.8773282141428607), (4.0, 0.8773282141428607), (11.0, 0.6289396137402194), (11.0, 0.6289396137402194), (13.0, 0.3835696840366291), (2.0, 0.6471414524498166), (0.0, 0.39970661066141056)]
fuzzy :  None
pembilang :  27.135986012803524
penyebut :  4.442953402914016
rata-rata tertimbang :  6.107645872451813
 
[11.0, 13.0, 11.0, 4.0, 4.0, 6.107645872451813, 2.0]
mean :  7.301092267493116
standar deviasi :  3.985002240797779
fuzzy :  [(11.0, 0.6500281376552196), 

[0.0, 14.0, 14.0, 14.0, 11.0, 4.0, 14.0]
mean :  10.142857142857142
standar deviasi :  5.3566666455007645
fuzzy :  [(0.0, 0.16654522562372726), (14.0, 0.771653851456372), (14.0, 0.771653851456372), (14.0, 0.771653851456372), (11.0, 0.9872806350894773), (4.0, 0.5181601360202222), (14.0, 0.771653851456372)]
fuzzy :  None
pembilang :  56.14534321162197
penyebut :  4.758601402558915
rata-rata tertimbang :  11.798706901870387
 
[0.0, 2.0, 18.0, 21.0, 23.0, 23.0, 0.0]
mean :  12.428571428571429
standar deviasi :  10.321366781821967
fuzzy :  [(0.0, 0.4843609775326966), (2.0, 0.6002628187858523), (18.0, 0.8644389895571871), (21.0, 0.7083677969129465), (23.0, 0.5918709319564519), (23.0, 0.5918709319564519), (0.0, 0.4843609775326966)]
fuzzy :  None
pembilang :  58.86221405476974
penyebut :  4.325533424234282
rata-rata tertimbang :  13.608082121152417
 
[0.0, 11.0, 9.0, 14.0, 11.0, 11.0, 11.0]
mean :  9.571428571428571
standar deviasi :  4.135461378894322
fuzzy :  [(0.0, 0.06869195110897694), (11

[0.0, 10.775556169757486, 11.0, 11.97591176490613, 13.0, 14.0, 10.370225182774854]
mean :  10.160241873919782
standar deviasi :  4.317126014144178
fuzzy :  [(0.0, 0.06271643632536308), (10.775556169757486, 0.9898952396888642), (11.0, 0.9812611587951536), (11.97591176490613, 0.9153653800068728), (13.0, 0.8054768863865883), (14.0, 0.6733452641925415), (10.370225182774854, 0.9988179162195472)]
fuzzy :  None
pembilang :  62.678879457309556
penyebut :  5.426878281614931
rata-rata tertimbang :  11.54971167672063
 
[0.0, 4.0, 2.0, 4.0, 4.0, 16.0, 14.0]
mean :  6.285714285714286
standar deviasi :  5.699982098074287
fuzzy :  [(0.0, 0.5444510951024146), (4.0, 0.9227530113493233), (2.0, 0.7537956199116784), (4.0, 0.9227530113493233), (4.0, 0.9227530113493233), (16.0, 0.23407569573734852), (14.0, 0.4002226792487221)]
fuzzy :  None
pembilang :  21.928956017294922
penyebut :  4.700804124048134
rata-rata tertimbang :  4.6649371976832406
 
[0.0, 4.0, 16.0, 14.0, 0.0, 21.0, 0.0]
mean :  7.8571428571428

[0.0, 4.0, 3.0, 11.0, 2.0, 2.0, 2.0]
mean :  3.4285714285714284
standar deviasi :  3.288818409491811
fuzzy :  [(0.0, 0.580805186972608), (4.0, 0.9850205504988865), (3.0, 0.9915462499161207), (11.0, 0.07067062808931716), (2.0, 0.9099825630789706), (2.0, 0.9099825630789706), (2.0, 0.9099825630789706)]
fuzzy :  None
pembilang :  13.15199323920022
penyebut :  5.3579903047138435
rata-rata tertimbang :  2.4546504363080652
 
[0.0, 7.0, 2.4546504363080652, 4.0, 2.0, 3.0, 11.0]
mean :  4.2078072051868665
standar deviasi :  3.40348179772903
fuzzy :  [(0.0, 0.4657204907330408), (7.0, 0.7142748249084431), (2.4546504363080652, 0.8757682524448189), (4.0, 0.9981379357811871), (2.0, 0.8102778722139435), (3.0, 0.938979858675226), (11.0, 0.13654198486492333)]
fuzzy :  None
pembilang :  17.08163759442
penyebut :  4.939701219621583
rata-rata tertimbang :  3.4580305234996738
 
[0.0, 2.0, 2.0, 2.0, 2.0, 14.0, 14.0]
mean :  5.142857142857143
standar deviasi :  5.642405045180428
fuzzy :  [(0.0, 0.660114782282

...

[0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0]
mean :  6.142857142857143
standar deviasi :  7.259054866235715
fuzzy :  [(0.0, 0.6990600402333105), (0.0, 0.6990600402333105), (0.0, 0.6990600402333105), (0.0, 0.6990600402333105), (11.0, 0.7994476203799281), (16.0, 0.3977768424039697), (16.0, 0.3977768424039697)]
fuzzy :  None
pembilang :  21.52278278110624
penyebut :  4.39124146612111
rata-rata tertimbang :  4.901297946641463
 
[0.0, 0.0, 2.0, 2.0, 2.0, 4.0, 4.0]
mean :  2.0
standar deviasi :  1.5118578920369088
fuzzy :  [(0.0, 0.41689984067442165), (0.0, 0.41689984067442165), (2.0, 1.0), (2.0, 1.0), (2.0, 1.0), (4.0, 0.41689984067442165), (4.0, 0.41689984067442165)]
fuzzy :  None
pembilang :  9.335198725395372
penyebut :  4.667599362697686
rata-rata tertimbang :  2.0
 
[0.0, 0.0, 0.0, 4.901297946641463, 11.0, 16.0, 16.0]
mean :  6.843042563805923
standar deviasi :  6.858072416100345
fuzzy :  [(0.0, 0.6078912793073392), (0.0, 0.6078912793073392), (0.0, 0.6078912793073392), (4.901297946641463, 0.

[0.0, 0.0, 1.281821155266385, 2.0, 2.0, 2.0, 2.0]
mean :  1.3259744507523408
standar deviasi :  0.8730576617966347
fuzzy :  [(0.0, 0.31562113260509844), (0.0, 0.31562113260509844), (1.281821155266385, 0.9987221268958151), (2.0, 0.742313782594539), (2.0, 0.742313782594539), (2.0, 0.742313782594539), (2.0, 0.742313782594539)]
fuzzy :  None
pembilang :  7.218693411244006
penyebut :  4.599219522484167
rata-rata tertimbang :  1.569547479948291
 
[0.0, 11.0, 16.0, 16.0, 26.0, 28.0, 58.0]
mean :  22.142857142857142
standar deviasi :  17.007801331225885
fuzzy :  [(0.0, 0.42852140167129366), (11.0, 0.8068672981726092), (16.0, 0.9368629265407349), (16.0, 0.9368629265407349), (26.0, 0.9746143413704176), (28.0, 0.9424310040282641), (58.0, 0.10837218159510632)]
fuzzy :  None
pembilang :  96.86878145014063
penyebut :  5.13453207991916
rata-rata tertimbang :  18.86613618191003
 
[11.0, 16.0, 16.0, 18.86613618191003, 26.0, 28.0, 58.0]
mean :  24.838019454558577
standar deviasi :  14.611348923056658
fu

[0.0, 16.0, 26.0, 27.0, 70.0, 0.0, 13.297831980854765]
mean :  21.756833140122108
standar deviasi :  22.11758775719719
fuzzy :  [(0.0, 0.6164536770231978), (16.0, 0.9666970509135611), (26.0, 0.9817677535088691), (27.0, 0.9722955051951422), (70.0, 0.09268041305115872), (0.0, 0.6164536770231978), (13.297831980854765, 0.9294812340770262)]
fuzzy :  None
pembilang :  86.09280723981135
penyebut :  5.175829310792152
rata-rata tertimbang :  16.633625660780336
 
[0.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0]
mean :  9.428571428571429
standar deviasi :  3.849198167230708
fuzzy :  [(0.0, 0.049802557184332984), (11.0, 0.9200523641748719), (11.0, 0.9200523641748719), (11.0, 0.9200523641748719), (11.0, 0.9200523641748719), (11.0, 0.9200523641748719), (11.0, 0.9200523641748719)]
fuzzy :  None
pembilang :  60.72345603554154
penyebut :  5.570116742233564
rata-rata tertimbang :  10.901648716825997
 
[0.0, 64.0, 81.0, 0.0, 64.92580706456063, 74.0, 70.0]
mean :  50.560829580651514
standar deviasi :  32.4115896

[0.0, 64.0, 81.0, 61.121625237690104, 74.0, 64.92580706456063, 70.0]
mean :  59.29249032889295
standar deviasi :  25.000050368143917
fuzzy :  [(0.0, 0.06007409355128636), (64.0, 0.9824295837940223), (81.0, 0.6859625971020723), (61.121625237690104, 0.9973272787886616), (74.0, 0.8411123533899055), (64.92580706456063, 0.9749348103545709), (70.0, 0.9123690313113917)]
fuzzy :  None
pembilang :  368.8033036417865
penyebut :  5.454209748291911
rata-rata tertimbang :  67.61810063452074
 
[0.0, 62.0, 33.0, 28.0, 28.0, 28.0, 49.0]
mean :  32.57142857142857
standar deviasi :  17.96709009635707
fuzzy :  [(0.0, 0.19339492679885206), (62.0, 0.2615200948321318), (33.0, 0.9997155839220748), (28.0, 0.968153304267344), (28.0, 0.968153304267344), (28.0, 0.968153304267344), (49.0, 0.6583667906204361)]
fuzzy :  None
pembilang :  162.78971044787892
penyebut :  5.017457308975526
rata-rata tertimbang :  32.44466278899294
 
[0.0, 0.0, 4.901297946641463, 6.082461241789016, 7.366495193489067, 11.0, 16.0]
mean : 

[0.0, 25.0, 28.0, 28.091707487165866, 32.0, 52.0, 52.0]
mean :  31.013101069595127
standar deviasi :  16.464060269369572
fuzzy :  [(0.0, 0.1696614366593823), (25.0, 0.9354869714013815), (28.0, 0.9833946713170276), (28.091707487165866, 0.9843822812563993), (32.0, 0.998205239860146), (52.0, 0.4438101951783374), (52.0, 0.4438101951783374)]
fuzzy :  None
pembilang :  156.67403215658686
penyebut :  4.958750990851012
rata-rata tertimbang :  31.59546273762352
 
[0.0, 77.0, 87.0, 64.0, 57.0, 73.0, 89.0]
mean :  63.857142857142854
standar deviasi :  28.15481979255098
fuzzy :  [(0.0, 0.07639734307353391), (77.0, 0.8967815971850329), (87.0, 0.7133413965344344), (64.0, 0.9999871287539164), (57.0, 0.9707798690210929), (73.0, 0.9486446822819521), (89.0, 0.6711904953652086)]
fuzzy :  None
pembilang :  379.43352915028237
penyebut :  5.277122512215171
rata-rata tertimbang :  71.90159566543927
 
[0.0, 29.0, 0.0, 13.297831980854765, 14.0, 14.0, 14.0]
mean :  12.042547425836394
standar deviasi :  9.185949

In [15]:
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
0,2.000000,50.000000,8642.109393,98.000000
1,6.690629,13.000000,3250.000000,28.000000
2,1.000000,16.000000,4000.000000,64.925807
3,2.000000,20.000000,5000.000000,45.000000
4,1.000000,24.000000,6000.000000,77.000000
...,...,...,...,...
743,23.000000,2.000000,500.000000,22.515604
744,7.265844,2.000000,542.761993,52.000000
745,5.005763,6.659449,1649.679157,62.000000
746,39.000000,1.000000,250.000000,24.493941


In [16]:
dummy.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
count,748.000000,748.000000,748.000000,748.000000
mean,9.104429,5.498295,1345.549092,33.669391
std,7.762147,5.839125,1409.375113,24.595888
min,1.000000,0.640911,89.772356,0.369338
25%,2.946764,2.000000,500.000000,14.000000
50%,7.000000,4.000000,1000.000000,28.000000
75%,14.000000,7.000000,1750.000000,50.000000
max,74.000000,50.000000,11500.000000,98.000000


# Klasifikasi

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [35]:
x = dummy.values
y = df.iloc[:, -1].values

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=72)

In [37]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [38]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[131  15]
 [ 30  11]]
0.7593582887700535
